In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/goodreadsbooks/books.csv
/kaggle/input/goodbooks-10k/book_tags.csv
/kaggle/input/goodbooks-10k/ratings.csv
/kaggle/input/goodbooks-10k/books.csv
/kaggle/input/goodbooks-10k/sample_book.xml
/kaggle/input/goodbooks-10k/to_read.csv
/kaggle/input/goodbooks-10k/tags.csv
/kaggle/input/book-datasets/Final Report.ipynb
/kaggle/input/book-datasets/.gitignore
/kaggle/input/book-datasets/README.md
/kaggle/input/book-datasets/books10k.csv
/kaggle/input/book-datasets/ratings10k.csv
/kaggle/input/book-datasets/Final Presentation.key
/kaggle/input/book-datasets/Code/EDA - full Goodreads authors, works, series, genres, interactions.ipynb
/kaggle/input/book-datasets/Code/Collab filtering using Surprise (algo list, cv, plot, user-based recommendations).ipynb
/kaggle/input/book-datasets/Code/Collab filtering using pearsonR (item-based recommendations).ipynb
/kaggle/input/book-datasets/Code/Data prep - full Goodreads loading files, statistics, distributions.ipynb
/kaggle/input/book-datasets/

##credit to https://github.com/Reinalynn/Building-a-Book-Recommendation-System-using-Python/blob/master/Final%20Report.ipynb

import gzip
def parse(path):
    g = gzip.open(path, 'r')
    for l in g:
        yield eval(l)
books = parse('Unused data/goodreads_books.json.gz')
next(books)

Ref: https://www.youtube.com/watch?v=xySjbVUgAwU

In [2]:
df = pd.read_csv("/kaggle/input/book-datasets/books10k.csv")


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 23 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   book_id                    10000 non-null  int64  
 1   goodreads_book_id          10000 non-null  int64  
 2   best_book_id               10000 non-null  int64  
 3   work_id                    10000 non-null  int64  
 4   books_count                10000 non-null  int64  
 5   isbn                       9300 non-null   object 
 6   isbn13                     9415 non-null   float64
 7   authors                    10000 non-null  object 
 8   original_publication_year  9979 non-null   float64
 9   original_title             9415 non-null   object 
 10  title                      10000 non-null  object 
 11  language_code              8916 non-null   object 
 12  average_rating             10000 non-null  float64
 13  ratings_count              10000 non-null  int6

In [4]:
df.head()

,book_id,goodreads_book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,...,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url
0,1,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,...,4780653,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...
1,2,3,3,4640799,491,439554934,9.780440e+12,"J.K. Rowling, Mary GrandPré",1997.0,Harry Potter and the Philosopher's Stone,...,4602479,4800065,75867,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...
2,3,41865,41865,3212258,226,316015849,9.780316e+12,Stephenie Meyer,2005.0,Twilight,...,3866839,3916824,95009,456191,436802,793319,875073,1355439,https://images.gr-assets.com/books/1361039443m...,https://images.gr-assets.com/books/1361039443s...
3,4,2657,2657,3275794,487,61120081,9.780061e+12,Harper Lee,1960.0,To Kill a Mockingbird,...,3198671,3340896,72586,60427,117415,446835,1001952,1714267,https://images.gr-assets.com/books/1361975680m...,https://images.gr-assets.com/books/1361975680s...
4,5,4671,4671,245494,1356,743273567,9.780743e+12,F. Scott Fitzgerald,1925.0,The Great Gatsby,...,2683664,2773745,51992,86236,197621,606158,936012,947718,https://images.gr-assets.com/books/1490528560m...,https://images.gr-assets.com/books/1490528560s...


In [5]:
#Create a list of columns to keep that are important
columns = ['title', 'authors']

In [6]:
#concatenate the data from all two columns together 
df['combined_features'] = df[columns].apply(lambda row: '_'.join(row.values.astype(str)), axis=1)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 24 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   book_id                    10000 non-null  int64  
 1   goodreads_book_id          10000 non-null  int64  
 2   best_book_id               10000 non-null  int64  
 3   work_id                    10000 non-null  int64  
 4   books_count                10000 non-null  int64  
 5   isbn                       9300 non-null   object 
 6   isbn13                     9415 non-null   float64
 7   authors                    10000 non-null  object 
 8   original_publication_year  9979 non-null   float64
 9   original_title             9415 non-null   object 
 10  title                      10000 non-null  object 
 11  language_code              8916 non-null   object 
 12  average_rating             10000 non-null  float64
 13  ratings_count              10000 non-null  int6

In [8]:
from sklearn.metrics.pairwise import cosine_similarity

In [9]:
from sklearn.feature_extraction.text import CountVectorizer

In [10]:
#convert text from the new column to a matrix of word counts
cm = CountVectorizer().fit_transform(df['combined_features'])

In [11]:
#cosine similarity function to get the similarity scores of each book. 
#A value of 1 means the current book at that row has a 100% similarity score to the book at that column.

#Get the cosine similarity matrix from the count matrix
# This will give us a nxn matrix of similarity scores for each book (row of data) to every other book 
#in the data set (the columns) including itself.

cs = cosine_similarity(cm)
#Print the similarity score
print(cs)

[[1.         0.13363062 0.         ... 0.26726124 0.25928149 0.23904572]
 [0.13363062 1.         0.         ... 0.125      0.12126781 0.1118034 ]
 [0.         0.         1.         ... 0.         0.         0.        ]
 ...
 [0.26726124 0.125      0.         ... 1.         0.24253563 0.2236068 ]
 [0.25928149 0.12126781 0.         ... 0.24253563 1.         0.21693046]
 [0.23904572 0.1118034  0.         ... 0.2236068  0.21693046 1.        ]]


In [12]:
#we can get the title of the book that the user likes.
Title = df['title'][1]
#Show the title
Title


"Harry Potter and the Sorcerer's Stone (Harry Potter, #1)"

In [13]:
#find the book_id the the user likes
book_id = df[df.title == Title]['book_id'].values[0]
#show the books_id
book_id

2

In [14]:
#create list of book_ids and similarity scores
scores = list(enumerate (cs[book_id]))
#print (scores)

In [15]:
#Sort the list of similar books in descending order
sorted_scores = sorted(scores, key= lambda x:x[1], reverse= True)
sorted_scores = sorted_scores[1:]
#Show the sorted scores
#sorted_scores

In [16]:
#create a loop tp print first 5 books from the sorted list
j = 0
print ('suggestions'+Title+' are:\n')

for item in sorted_scores:
    book_title = df[df.book_id == item [0]] ['title'].values[0]
    print (j+1, book_title)
    j=j+1
    
    if j >= 6:
        break


suggestionsHarry Potter and the Sorcerer's Stone (Harry Potter, #1) are:

1 How to Cook Everything: Simple Recipes for Great Food
2 City of Bones (The Mortal Instruments, #1)
3 A Voice in the Wind (Mark of the Lion, #1)
4 Fahrenheit 451
5 Brave New World
6 The Chamber
